In [ ]:
import tensorflow as tf
import loadIvs_test
import numpy as np
import os

batch_size = 32

iv_params = {
    "short_ivs_dir": "/work/jiacen/ivectors",
    "long_ivs_dir": "/work/jiacen/long_ivectors",
    "task_type": "test",
    "batch_size": batch_size,
}

ivectors = loadIvs_test.IvectorLoader(**iv_params)

In [ ]:
def glorot_init(shape, dtype, partition_info=None):
    with tf.name_scope("glorot_init"):
        return tf.random_normal(shape=shape, stddev=1./tf.sqrt(shape[0]/2.), dtype=dtype)

def lrelu(x, a=0.2):
    with tf.name_scope("leaky_relu"):
        return tf.nn.relu(tf.maximum(x, a*x))


In [ ]:
sess = tf.Session()
saver = tf.train.import_meta_graph("/work/jiacen/wooo/tfmodels/cgan-model-1000000.meta")
saver.restore(sess, tf.train.latest_checkpoint("/work/jiacen/wooo/tfmodels/."))

In [ ]:
noise_dim=400
input_dim = 400

In [ ]:
tf.trainable_variables()

In [ ]:
graph = tf.get_default_graph()

In [ ]:
def generator_test_all(x, z):
    input_data = x+z
    layers = [input_data]
    for i in range(0, 5):
        kernel = graph.get_tensor_by_name("generator/gen%d/kernel:0" % (i,))
        bias = graph.get_tensor_by_name("generator/gen%d/bias:0" % (i,))
        layer = lrelu(tf.add(tf.matmul(layers[-1], kernel), bias))
        layers.append(layer)
    return layers
def generator(x, z):
    input_data = x+z
    layers = [input_data]
    for i in range(0, 5):
        kernel = graph.get_tensor_by_name("generator/gen%d/kernel:0" % (i,))
        bias = graph.get_tensor_by_name("generator/gen%d/bias:0" % (i,))
        layer = lrelu(tf.add(tf.matmul(layers[-1], kernel), bias))
        layers.append(layer)
    return layers[-1]

In [ ]:
batch_data = ivectors.next_batch()
priori = batch_data[:, 0, :]
truth = batch_data[:, 1, :]

In [ ]:
noise = np.random.normal(-1., 1., size=[32, 400])

In [ ]:
input_random = tf.placeholder(tf.float32, shape=[None, noise_dim], name="input_random")
input_ivector = tf.placeholder(tf.float32, shape=[None, input_dim], name="input_ivector")
gen_sample_all = generator_test_all(input_ivector, input_random)
gen_sample = generator(input_ivector, input_random)

In [ ]:
generated = sess.run(gen_sample_all, feed_dict={input_random:noise, input_ivector: priori})
for i in range(5):
    print generated[i][0]

In [ ]:
def discriminator_test(x):
    layers = [x]
    for i in range(2):
        kernel = graph.get_tensor_by_name("discriminator/disc%d/kernel:0" % (i,))
        bias = graph.get_tensor_by_name("discriminator/disc%d/bias:0" % (i,))
        layer = lrelu(tf.add(tf.matmul(layers[-1], kernel), bias))
        layers.append(layer)
    kernel = graph.get_tensor_by_name("discriminator/disc%d/kernel:0" % (2,))
    bias = graph.get_tensor_by_name("discriminator/disc%d/bias:0" % (2,))
    layer = tf.sigmoid(tf.add(tf.matmul(layers[-1], kernel), bias))
    layers.append(layer)
    return layers
disc_fake = discriminator_test(gen_sample)
res = sess.run(disc_fake, feed_dict={input_random:noise, input_ivector: priori})

In [ ]:
disc0k=sess.run(graph.get_tensor_by_name("discriminator/disc%d/kernel:0" % (0,)))
disc0b=sess.run(graph.get_tensor_by_name("discriminator/disc%d/bias:0" % (0,)))

In [ ]:
disc0k

In [ ]:
res_real